In [1]:
zmq_port = 14000
osc_port = 14001

In [2]:
import sys, os
lib_root = '/root/shared/ml-examples/apple2373/chainer_caption_generation'

sys.path.append(os.path.join(lib_root, 'codes'))
sys.path.append(os.curdir)

import numpy as np
from caption_generator import Caption_generator
from image_reader import Image_reader

import zmq
from OSC import OSCClient, OSCMessage
from cStringIO import StringIO

#Instantiate image_reader with GoogleNet mean image
mean_image = np.array([104, 117, 123]).reshape((3,1,1))
image_reader=Image_reader(mean=mean_image)

#Instantiate caption generator
caption_model=os.path.join(lib_root, 'models/caption_model.chainer')
cnn_model=os.path.join(lib_root,'data/bvlc_googlenet_caffe_chainer.pkl')
index2word=os.path.join(lib_root,'work/index2token.pkl')
generator=Caption_generator(caption_model_place=caption_model,cnn_model_place=cnn_model,index2word_place=index2word)

#Specify IP address
ip = '.'.join(map(lambda (i,x): x if i!=3 else '1' , enumerate(os.environ['HOST_IP'].split('.'))))

#Instantiate zmq subscriber and osc client
context = zmq.Context()
subs = context.socket(zmq.SUB)
subs.setsockopt(zmq.SUBSCRIBE, '')
subs.connect("tcp://" + ip + ":" + str(zmq_port))

client = OSCClient()
client.connect((ip, osc_port))

/usr/local/lib/python2.7/dist-packages/chainer/functions/activation/lstm.py:15: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + numpy.exp(-x))


In [3]:
try:
    print 'running video captioning ...'
    sys.stdout.flush()
    while True:
        buf = subs.recv()
        buf_str_io = StringIO(buf)
        image = image_reader.read(buf_str_io)
        captions = generator.generate(image)

        m = OSCMessage()
        m.setAddress("/caption")
        m.append(' '.join(map(str,captions[0]['sentence'])))
        client.send(m)
except:
    print 'finish'

running video captioning ...
finish
